## 스트리밍 서비스를 제공하는 스타트업 metflix에서 10개월 이상 가입 중인 고객 정보를 이용하여 마케팅 전략을 수립하려한다.

metflix.csv

|컬럼|정의|type|
|:---|:---|:---:|
|ids|식별 번호|int64|
|ages|나이|int64|
|bookmarks|즐겨찾기 수|int64|
|replies|답글 수|int64|
|ratings|별점 수|int64|
|watches|시청 횟수|int64|
|period|가입 기간([가입 후 경과한 월 수]M)|object|  

정답 및 해설 : https://tjd229.tistory.com/23

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('../content/tjd229/metflix.csv')

df.shape

(540, 7)

### 분석을 수행하기 전, 아래의 전처리를 수행하시오.

단계 1 : 가입 기간(period) 컬럼에서 문자 M을 제거한 후 정수형으로 변경하시오

상기 전처리를 완료한 데이터셋(데이터셋명: <span style="color:skyblue">customer</span>)을 이용하여 다음 1~3번 문제에 답하시오.

In [3]:
df.period.unique()

array(['16M', '10M', '21M', '22M', '24M', '13M', '27M', '19M', '17M',
       '15M', '18M', '36M', '34M', '12M', '31M', '11M', '33M', '20M',
       '32M', '25M', '14M', '30M', '26M', '23M', '35M', '29M', '28M'],
      dtype=object)

 👉 df.series.str.replace({'A':'B', 'C':'D'})

  relace 은 2개 이상 
  where 은 조건으로 2개로 분류

In [4]:
df.period = df.period.str.replace('M','').astype(int)
df.info()

# df['period'] = df['period'].str.slice(0,2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   ids        540 non-null    int64
 1   ages       540 non-null    int64
 2   bookmarks  540 non-null    int64
 3   replies    540 non-null    int64
 4   ratings    540 non-null    int64
 5   watches    540 non-null    int64
 6   period     540 non-null    int64
dtypes: int64(7)
memory usage: 29.7 KB


In [5]:
customer = df.copy()

### Q1. (<span style="color:skyblue">customer</span>를 이용하여)별점 수(ratings) 컬럼에서 이상치를 제외한 별점 수(ratings) 평균을 구하시오.  
이상치는 아래 구간에 포함되지 않은 데이터로 간주한다.

$ [ Median - 2.3 \times IQR , Median + 2.3 \times IQR ]  $

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

 👉 IQR : quantile(0.25, 0.75)

In [6]:
# IQR = Q3 - Q1 
Q1 = customer.ratings.quantile(0.25)
Q3 = customer.ratings.quantile(0.75)
IQR = Q3-Q1
IQR



190.5

In [7]:
uplimit = customer.ratings.median()+(2.3*IQR)
underlimit = customer.ratings.median()-(2.3*IQR)
uplimit, underlimit

(537.65, -338.65)

In [8]:
rating_1 = customer.loc[(customer.ratings > underlimit) & (customer.ratings < uplimit), 'ratings' ]
rating_1.mean().round(2)

126.77

In [9]:
# customer.reset_index(inplace=True)
# customer.info()

### Q2. (<span style="color:skyblue">customer</span>를 이용하여)고객 나이대 별 활동 빈도를 분석하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 고객 데이터의 ages 컬럼을 이용하여 아래 규칙에 따라 age_cls 컬럼을 새로 생성한다.
- ages가 10이상 19이하인 경우 age_cls는 10s가 된다.  
- ages가 20이상 29이하인 경우 age_cls는 20s가 된다.  
- ages가 30이상 39이하인 경우 age_cls는 30s가 된다.  
- ages가 40이상 49이하인 경우 age_cls는 40s가 된다.  
- ages가 50이상인 경우 age_cls는 50s가 된다.  

단계 2 : age_cls별 답글 수(replies)의 평균을 구한다.  
단계 3 : 단계 2에서 구한 age_cls별 답글 수(replies)의 평균에서 최대값을 A, 최소값을 B라고 할 때, A+B를 구하시오 


※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [10]:
# default 값을 셋팅한다.
customer['age_cls'] = np.nan

 👉 cut() / where()

 cut 은 여러개가 있을때,
 where 은 2개가 있을때, 

In [11]:

customer['age_cls'] = pd.cut(customer.ages, 
                             bins = [10,19,29,39,49,100],
                             labels= ['10s','20s','30s','40s','50s'])
customer.loc[customer.ages ==50,['ages','age_cls']]

,ages,age_cls
154,50,50s
200,50,50s
268,50,50s
358,50,50s
487,50,50s


In [12]:
# print(customer.groupby('age_cls').mean()) 
customer.groupby('age_cls').replies.mean().sort_values()

#customer.groupby('age_cls').replies 는 series 라서 sort_values() 을 사용

age_cls
40s     6.482456
50s     7.416667
10s    68.135135
20s    69.344086
30s    75.804469
Name: replies, dtype: float64

In [13]:
customer.groupby('age_cls')[['replies']].mean().sort_values(by='replies')

#customer.groupby('age_cls')[['replies']] 는 df type 이라서 sort_values(by='replies') 을 사용 

,replies
age_cls,
40s,6.482456
50s,7.416667
10s,68.135135
20s,69.344086
30s,75.804469


In [14]:
customer.loc[:,['replies','age_cls']].groupby('age_cls').mean().sort_values('replies')

,replies
age_cls,
40s,6.482456
50s,7.416667
10s,68.135135
20s,69.344086
30s,75.804469


In [15]:
rep_max = customer.groupby('age_cls').replies.mean().max()
rep_min = customer.groupby('age_cls').replies.mean().min()
rep_max,rep_min


(75.80446927374301, 6.482456140350878)

In [16]:

round(rep_max+rep_min,2)

82.29

In [17]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ids        540 non-null    int64   
 1   ages       540 non-null    int64   
 2   bookmarks  540 non-null    int64   
 3   replies    540 non-null    int64   
 4   ratings    540 non-null    int64   
 5   watches    540 non-null    int64   
 6   period     540 non-null    int64   
 7   age_cls    540 non-null    category
dtypes: category(1), int64(7)
memory usage: 30.4 KB


### Q3. (<span style="color:skyblue">customer</span>를 이용하여)고객 나이대를 예측하기 위한 모델을 만드려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 고객 데이터의 ages 컬럼을 이용하여 아래 규칙에 따라 age_cls 컬럼을 새로 생성한다.  
- ages가 40이상인 경우 age_cls는 1이 되고, 그 외에는 0이 된다. 


In [35]:
df3 = customer.copy()

In [19]:
df3.where(df3.ages >= 40).head()

# 40 큰 값은 그대로 유지되어 가져오고, 그외 값은 NaN

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
0,912.0,47.0,5.0,11.0,23.0,74.0,16.0,40s
1,325.0,44.0,6.0,4.0,11.0,90.0,10.0,40s
2,282.0,49.0,6.0,12.0,38.0,188.0,10.0,40s
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


👉 np.where

df.where, series.where 은 필터링한 값을 가져온다.

np.where은 해당 원소 값 대신 위치를 가져온다는 점과 값이 True인 위치에도 대치할 값 지정이 가능하다는 점

```df.series = np.where(df.series 조건, true, false)```

In [36]:
df3.age_cls = np.where(df3.ages >= 40,1,0)
df3.head()
# df3.where(df3.ages >= 40,1)   # -> 40 큰 값은 그대로 유지되어 가져오고, 그외 값은 (NaN) 은 1로 대체 됨
# df3 = np.where(df3.ages >= 40,1,0) # -> 0,1 이 채워진 리스트로 df3 가 업데이트 됨.


,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
0,912,47,5,11,23,74,16,1
1,325,44,6,4,11,90,10,1
2,282,49,6,12,38,188,10,1
3,1234,21,13,53,89,766,21,0
4,1279,25,40,73,186,736,22,0


단계 2 : Train Set과 Test Set을 99:1 비율로 나눈다.  
- Train Set에서 답글 수(replies)가 0인 데이터는 총 13개이다.
- Test Set에서 답글 수(replies)가 0인 데이터는 존재하지 않는다.

In [37]:
from sklearn.model_selection import train_test_split  

Train, test = train_test_split(df3,test_size=0.01,random_state=229)


In [38]:
test.replies.value_counts()

41    1
94    1
22    1
68    1
91    1
8     1
Name: replies, dtype: int64

In [39]:
Train

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls
336,195,48,2,7,20,190,10,1
278,1334,24,15,0,0,647,23,0
149,500,49,0,4,6,84,10,1
241,2130,23,46,102,103,716,36,0
413,2140,42,2,1,3,80,36,1
...,...,...,...,...,...,...,...,...
50,182,37,6,81,234,841,10,0
501,406,24,26,71,81,309,10,0
112,360,42,5,11,36,181,10,1
476,621,33,6,139,388,53,11,0


In [40]:

from sklearn.neighbors import KNeighborsClassifier

단계 3 : Train Set을 이용하여 KNeighborsClassifier를 학습한다. 
- 독립 변수 : 가입 기간(period)을 (replies+0.01)로 나눈 값
- 종속 변수 : age_cls


👉 Series.to_frame()

In [41]:
X_train_3 = (Train.period/(Train.replies+0.01)).to_frame()
X_test_3  = (test['period']/(test['replies']+0.01)).to_frame()

y_train_3 = Train['age_cls']
y_test_3 = test['age_cls']

👉 KNeighborsClassifier

>n_neighborsint (default=5)  
p (default=2) :  1 - manhattan_distance, 2 - euclidean_distance  
weights(default='uniform'): 'distance'로 설정하면, 분류할 때 인접한 샘플의 거리에 따라 다른 가중치 부여 (가까울수록 큰 가중치, 거리역순)

>knn.predict: 예측한 클래스 번호를 반환  
knn.predict_proba: 클래스 별 확률 값을 반환  
knn.kneighbors: 인접한 k개의 sample에 대해 거리, index를 반환  



KNeighborsRegressor

In [43]:
my_knn = KNeighborsClassifier(p=1,weights='distance')
my_knn.fit(X_train_3, y_train_3)
y_pred_3 = my_knn.predict(X_test_3)

test['pred'] = y_pred_3

test

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls,pred
503,2145,37,34,41,50,123,36,0,0
397,95,26,19,94,161,782,10,0,0
524,1241,35,20,22,70,831,21,0,0
32,635,24,49,68,187,430,12,0,0
341,855,21,26,91,303,456,15,0,0
225,513,52,0,8,29,74,10,1,1


In [60]:
A1 = test.loc[(test.age_cls == 0) & (test.pred == 0) , ['age_cls','pred']].shape[0]


In [61]:
B1 =test.loc[(test.age_cls == 1) & (test.pred == 0) , ['age_cls','pred']].shape[0]

In [62]:
answer1 = A1/A1+B1
answer1


1.0

👉 precision_score

In [70]:
from sklearn.metrics import precision_score, classification_report

precision_score(test.age_cls,test.pred,pos_label=0)

1.0

In [73]:
print(classification_report(test.age_cls,test.pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         1

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



단계 4 : Train Set을 이용하여 KNeighborsClassifier를 새로 학습한다. 
- 독립 변수 : 가입 기간(period)을 답글 수(replies)로 나눈 값, 나누기 전 답글 수(replies)가 0인 데이터는 제거한다.
- 종속 변수 : age_cls


In [44]:
Train2 = Train.loc[Train.replies != 0,].copy()

In [45]:
X_train_4 = (Train2.period/(Train2.replies)).to_frame()
X_test_4  = (test['period']/(test['replies'])).to_frame()

y_train_4 = Train2['age_cls']
y_test_4 = test['age_cls']

In [46]:
my_knn = KNeighborsClassifier(p=1,weights='distance')
my_knn.fit(X_train_4, y_train_4)
y_pred_4 = my_knn.predict(X_test_4)
test['pred2'] = y_pred_4
test

,ids,ages,bookmarks,replies,ratings,watches,period,age_cls,pred,pred2
503,2145,37,34,41,50,123,36,0,0,0
397,95,26,19,94,161,782,10,0,0,0
524,1241,35,20,22,70,831,21,0,0,0
32,635,24,49,68,187,430,12,0,0,0
341,855,21,26,91,303,456,15,0,0,0
225,513,52,0,8,29,74,10,1,1,1


In [63]:
A2 = test.loc[(test.age_cls == 0) & (test.pred2 == 0) , ['age_cls','pred2']].shape[0]

In [67]:
B2 =test.loc[(test.age_cls == 1) & (test.pred2 == 0) , ['age_cls','pred2']].shape[0]

,age_cls,pred2


In [65]:
answer2 = A2/A2+B2
answer2

1.0

In [75]:
print(classification_report(test.age_cls,test.pred2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         1

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



classification_report


단계 5 : 단계 4와 단계 5에서 학습한 두 모델을 각각 Test Set에 적용하여 age_cls를 예측한다. Test Set에 주어진 종속 변수를 활용하여 아래 정의된 지표 P를 각각 구하고, 두 값중 가장 큰 값을 구하시오
- 예측값 $ \hat{y_i} $도 0이고 실제값 $ y_i $도 0인 경우를 A라고 한다. 
- 예측값 $ \hat{y_i} $도 0이고 실제값 $ y_i $도 1인 경우를 B라고 한다. 

$$ P={A \over (A+B)} $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

👉 분류 성능평가  
https://datascienceschool.net/03%20machine%20learning/09.04%20%EB%B6%84%EB%A5%98%20%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80.html

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.model_selection import train_test_split  
from sklearn.neighbors import KNeighborsClassifier    
KNeighborsClassifier의 거리는 Manhattan Distance 사용, 가중치는 거리 역순으로 설정  
모든 random_state = 229  
문제 지시 외 Default 값 사용  